In [19]:
import requests # 웹 요청 도구
from bs4 import BeautifulSoup # html에서 데이터 읽는 도구
from bs4.element import NavigableString, Tag, Comment, ProcessingInstruction

In [32]:
response = requests.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findRealTicketList.do")

In [33]:
soup = BeautifulSoup(response.content)

In [34]:
table = soup.select_one('.tbl_comm.th_sort')

In [29]:
th_list = table.select('thead tr th')
columns = []
for th in th_list:
    for c in th.children:
        # print(c)
        if type(c) == NavigableString and len(c.strip()) > 0:
            columns.append(c.strip())

In [35]:
columns

['순위', '영화명', '개봉일', '예매율', '예매매출액', '누적매출액', '예매관객수', '누적관객수']

In [36]:
tr_list = table.select('tbody tr')
values_list = []
for tr in tr_list:
    for td in tr.select('td'):
        print(td)


#beautifulsoup만으로는 크롤링 불가능. selenium(+chrome webdriver) 설치 필수

<td colspan="8">검색된 데이터가 존재하지 않습니다.</td>


In [38]:
from selenium import webdriver

In [43]:
browser_proxy = webdriver.Chrome("/Users/attitude/workspace/tools/chromedriver")

/opt/anaconda3/envs/kdigital-pusan/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [45]:
browser_proxy.get("https://www.kobis.or.kr/kobis/business/stat/boxs/findRealTicketList.do")

In [46]:
soup = BeautifulSoup(browser_proxy.page_source)

In [47]:
table = soup.select_one('.tbl_comm.th_sort')

In [48]:
th_list = table.select('thead tr th')
columns = []
for th in th_list:
    for c in th.children:
        # print(c)
        if type(c) == NavigableString and len(c.strip()) > 0:
            columns.append(c.strip())

In [49]:
columns

['순위', '영화명', '개봉일', '예매율', '예매매출액', '누적매출액', '예매관객수', '누적관객수']

In [60]:
tr_list = table.select('tbody tr')
values_list = []
for tr in tr_list:
    values = []
    for td in tr.select('td'):
        values.append(td.text.strip().replace("%", '').replace(",", ""))
    values_list.append(values)


In [61]:
values_list[:3]

[['1',
  '듄',
  '2021-10-20',
  '16.2',
  '433676170',
  '15367958720',
  '26585',
  '1427720'],
 ['2',
  '고스트버스터즈 라이즈',
  '2021-12-01',
  '12.9',
  '186913270',
  '5115000',
  '21169',
  '545'],
 ['3',
  '유체이탈자',
  '2021-11-24',
  '11.4',
  '156212570',
  '3871617940',
  '18707',
  '398455']]

In [63]:
import pandas as pd

data = pd.DataFrame(values_list, columns=columns)

In [73]:
# data.info()
type(data.dtypes)
for column in data.columns[[0, 3, 4, 5, 6, 7]]:
    data[column] = data[column].astype(float)

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   순위      241 non-null    float64
 1   영화명     241 non-null    object 
 2   개봉일     241 non-null    object 
 3   예매율     241 non-null    float64
 4   예매매출액   241 non-null    float64
 5   누적매출액   241 non-null    float64
 6   예매관객수   241 non-null    float64
 7   누적관객수   241 non-null    float64
dtypes: float64(6), object(2)
memory usage: 15.2+ KB


In [76]:
data.head(10)

,순위,영화명,개봉일,예매율,예매매출액,누적매출액,예매관객수,누적관객수
0,1.0,듄,2021-10-20,16.2,433676170.0,1.536796e+10,26585.0,1427720.0
1,2.0,고스트버스터즈 라이즈,2021-12-01,12.9,186913270.0,5.115000e+06,21169.0,545.0
2,3.0,유체이탈자,2021-11-24,11.4,156212570.0,3.871618e+09,18707.0,398455.0
3,4.0,라스트 나잇 인 소호,2021-12-01,9.6,124623010.0,2.358000e+06,15739.0,262.0
4,5.0,엔칸토: 마법의 세계,2021-11-24,6.7,101574260.0,2.718332e+09,11009.0,284859.0
5,6.0,연애 빠진 로맨스,2021-11-24,5.7,88610600.0,2.338134e+09,9382.0,231656.0
6,7.0,태일이,2021-12-01,4.9,66347210.0,4.840010e+08,8083.0,58691.0
7,8.0,킬링 카인드: 킬러의 수제자,2021-12-01,3.5,45755930.0,3.756000e+06,5769.0,406.0
8,9.0,몬스타엑스 : 더 드리밍,2021-12-08,2.9,75984000.0,0.000000e+00,4749.0,0.0
9,10.0,귀멸의 칼날: 주합회의·나비저택 편,2021-12-01,2.6,37634370.0,0.000000e+00,4199.0,0.0
